# Missing values in crops

As an example of missing we'll be using the crops-dataset. You can find it in a zip-file in the files-folder. We'll use python to unpack this file.

In [6]:
import zipfile
import os

file = "files/food-twentieth-century-crop-statistics-1900-2017-xlsx.zip"
folder = "files/crop-statistics"


if not os.path.exists(folder):
    os.makedirs(folder)

with zipfile.ZipFile(file, 'r') as zip_ref:
    zip_ref.extractall(folder)
    
all_files = os.listdir(folder)
data_file = [f for f in all_files if f.endswith('.xlsx')][0]

And now it's up to you. Import the excel-file referenced in "data_file" into a pandas dataframe. (To read excel-files you'll need "openpyxl". Use pip install if you don't have it yet.)

In [ ]:
#DELETE
import pandas as pd

df = pd.read_excel(os.path.join(folder, data_file))
print(df.head())

If you simply imported the file you'll now notice that you only have 2 columns. The problem is the excel-file has multiple worksheets, and we need the second worksheet.

![](files/2024-10-05-15-09-49.png)

In [ ]:
#DELETE
# Import the second worksheet of "data_file" into a pandas dataframe
df = pd.read_excel(os.path.join(folder, data_file), sheet_name=1)
print(df.head())

Now show all columns, along with a count of the number of NaN-values they contain.

In [ ]:
#DELETE

# Show all columns along with a count of the number of NaN-values they contain
nan_counts = df.isna().sum()
print(nan_counts)

Drop "admin2" and "notes". They won't be of much help.

In [ ]:
#DELETE
# Drop "admin2" and "notes" columns from the dataframe
df = df.drop(columns=['admin2', 'notes'])
print(df.head())

As the field-names show there is a relationship between "hectares", "production" and "yield", being:

yield = production / hectares

That means that if we have two of these values we can calculate the third.

First drop all rows where we don't know any of the values or only one of them.

In [ ]:
#DELETE
# Drop all rows where all three of 'production (tonnes)', 'yield(tonnes/ha)', and 'hectares (ha)' are NaN

df = df.dropna(subset=['yield(tonnes/ha)', 'hectares (ha)'], how='all')
df = df.dropna(subset=['production (tonnes)', 'hectares (ha)'], how='all')
df = df.dropna(subset=['production (tonnes)', 'yield(tonnes/ha)'], how='all')
print(df.head())

And now you can start filling up. First make sure "yield" is equal to production divided by hectares.

In [ ]:
df['yield(tonnes/ha)'] = df.apply(
    lambda row: row['production (tonnes)'] / row['hectares (ha)'] if pd.isna(row['yield(tonnes/ha)']) else row['yield(tonnes/ha)'],
    axis=1
)
print(df.head())

You'll notice that this won't work as some values in the fields are 0. If two of the three values are zero the third will be zero as well, but if you only one value is zero that is wrong data (you can't get a yield if you don't have fields.)

So let's start at the beginning:

If hectares and production are zero, yield will be zero too.

In [ ]:
#DELETE

df.loc[(df['hectares (ha)'] == 0) & (df['production (tonnes)'] == 0), 'yield(tonnes/ha)'] = df['yield(tonnes/ha)'].fillna(0)

print(df.head())

Drop all rows where:
- Yield is NaN
- Hectares or production is 0

As these are the rows where you have a certain production without knowing the fields (or visa versa).

In [85]:
#DELETE

bad_rows = df.loc[(df['yield(tonnes/ha)'].isna()) & ((df['hectares (ha)'] == 0) | (df['production (tonnes)'] == 0))]

df.drop(bad_rows.index, inplace=True)


That takes care of all the rows with Na-values in yield. Looking back at the code you may realize that we didn't actually calculate any data, we simply dropped rows with too many zeros and set production to zero.

Next up is calculating hectares and production. They are:

- Zero if the other two are zero
- Dropped if only one of the others is not zero
- Calculated if both of the others have value

In [86]:
#DELETE
df.fillna({'hectares (ha)': df['production (tonnes)'] / df['yield(tonnes/ha)']}, inplace=True)
df.fillna({'production (tonnes)': df['yield(tonnes/ha)'] / df['hectares (ha)']}, inplace=True)


And finally, don't forget to delete the folder with the unzipped files.

In [88]:
import shutil
shutil.rmtree(folder)